<div align="center">
    <div><img src="../assets/redis_logo.svg" style="width: 130px"> </div>
    <div style="display: inline-block; text-align: center; margin-bottom: 10px;">
        <span style="font-size: 36px;"><b>Introduction to Redis Python</b></span>
        <br />
    </div>
    <br />
</div>

This notebook introduces [Redis](https://redis.io) and the standard Python client, [redis-py](https://redis-py.readthedocs.io/en/stable/), for interacting with the database. We will explore the basics of Redis setup, data structures, and capabilities like vector search!


### Install Python Dependencies

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["ROOT_DIR"] = parent_directory
print(dir_path)
print(parent_directory)

/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/getting_started
/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss


In [2]:
%pip install -r $ROOT_DIR/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [34]:
import os
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Hello World Redis

Now let's connect to the Redis db and get a basic feel for the most common
commands and data structures.

In [35]:
import redis
import json
import numpy as np

from time import sleep

# Connect with the Redis Python Client
client = redis.Redis.from_url(REDIS_URL)

client.ping()

True

In [36]:
client.dbsize() # should be empty

0

Redis, at it's core, is a simple key/value store. It supports a number of interesting
and flexible data structures that can solve a variatey of business and operational
problems.

### Strings

The basic string data type can be accessed using set/get methods. You can also place a
TTL policy (expiration) on any key in Redis.

In [37]:
client.set("hello", "world")

True

In [38]:
client.get("hello")

b'world'

In [39]:
client.delete("hello")

1

In [40]:
client.set("hello", "world")
client.expire("hello", time=3)

sleep(4)

# should be EMPTY
client.get("hello")

### Hashes
Hashes are collections of key/value pairs that are grouped together. It gets
serialized as a string in Redis, but can hold a variety of data in each field.

You can think of a Hash as a one-level deep Python dictionary.


In [41]:
obj = {
    "user": "john",
    "age": 45,
    "job": "dentist",
    "bio": "long form text of john's bio",
    "user_embedding": np.array([0.3, 0.4, -0.8], dtype=np.float32).tobytes() # cast vectors to bytes string
}

In [42]:
client.hset("user:john", mapping=obj)

5

In [43]:
client.hgetall("user:john")

{b'user': b'john',
 b'age': b'45',
 b'job': b'dentist',
 b'bio': b"long form text of john's bio",
 b'user_embedding': b'\x9a\x99\x99>\xcd\xcc\xcc>\xcd\xccL\xbf'}

In [44]:
client.delete("user:john")

1

### JSON
With the JSON capabilitie enabled, Redis can be a drop-in replacement for MongoDB
or other slower document databases. You can store nested and structured JSON data
directly in Redis.

In [45]:
# set a JSON obj
obj = {
    "user": "john",
    "metadata": {
        "age": 45,
        "job": "dentist",
    },
    "user_embedding": [0.3, 0.4, -0.8]
}

client.json().set("user:john", "$", obj)

True

In [46]:
# get user JSON obj
client.json().get("user:john")

{'user': 'john',
 'metadata': {'age': 45, 'job': 'dentist'},
 'user_embedding': [0.3, 0.4, -0.8]}

In [47]:
# grab array length for embedding field
client.json().arrlen("user:john", "$.user_embedding")

[3]

In [48]:
# grab obj keys
client.json().objkeys("user:john", "$")

[[b'user', b'metadata', b'user_embedding']]

In [49]:
# delete user JSON
client.delete("user:john")

1

### Lists
Lists store sequences of information... potentially list of messages in an LLM
converstion flow, or really any list of items in a queue.

In [50]:
# add items to a list
client.rpush("messages:john", *[
    json.dumps({"role": "user", "content": "Hello what can you do for me?"}),
    json.dumps({"role": "assistant", "content": "Hi, I am a helpful virtual assistant."})
])

2

In [51]:
# list all items in the list using indices
[json.loads(msg) for msg in client.lrange("messages:john", 0, -1)]

[{'role': 'user', 'content': 'Hello what can you do for me?'},
 {'role': 'assistant', 'content': 'Hi, I am a helpful virtual assistant.'}]

In [52]:
# count items in the list
client.llen("messages:john")

2

In [53]:
# pop the first item from the list and push to another list
client.rpoplpush("messages:john", "read_messages:john")

b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}'

In [54]:
client.lrange("read_messages:john", 0, -1)

[b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}']

In [55]:
# list cleanup
client.delete("messages:john", "read_messages:john")

2

### Pipelines
All Redis commands can be pipelined to gain some round trip latency improvements.

In [56]:
with client.pipeline(transaction=False) as pipe:
    for i in range(50):
        pipe.json().set(f"user:{i}", "$", obj)
    # execute batch
    pipe.execute()

In [57]:
client.dbsize()

50

In [58]:
# clean up!
client.flushall()

True

## Intro to Vector Search in Redis

Now that we have the basics down, we will dive into the fundamentals of vector
search in Redis using the base Python client.

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter


In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs from a folder
data_path = f"{parent_directory}/resources/10K"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

Listing available documents ... ['/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/nke-10k-2023.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/mu-10K-2019.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/amzn-10k-2023.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/jnj-10k-2023.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/amzn-10K-2019.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/aapl-10k-2023.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/aapl-10K-2019.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/nvd-10k-2023.pdf', '/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/msft-10k-2023.pdf']


In [60]:
# fetch the Nike PDF
doc = [doc for doc in docs if "nke" in doc][0]

# set up the file loader/extractor and text splitter to create chunks
loader = UnstructuredFileLoader(doc, mode="single", strategy="fast")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=0)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Done preprocessing. Created 180 chunks of the original pdf /Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/resources/10K/nke-10k-2023.pdf


In [61]:
# Take a look at content from a chunk
print(chunks[25].page_content)

NIKE is a consumer products company and the relative popularity of various sports and fitness activities and changing design trends affect the demand for our products, services and experiences. The athletic footwear, apparel and equipment industry is highly competitive both in the United States and worldwide. We compete internationally with a significant number of athletic and leisure footwear companies, athletic and leisure apparel companies, sports equipment companies, private labels and large companies that have diversified lines of athletic and leisure footwear, apparel and equipment. We also compete with other companies for the production capacity of contract manufacturers that produce our products. In addition, we and our contract manufacturers compete with other companies and industries for raw materials used in our products. Our NIKE Direct operations, both through our digital commerce operations and retail stores, also compete with multi-brand retailers, which sell our product

### Text embedding generation with SentenceTransformers

#### SentenceTransformer Models Cache folder
We are using `SentenceTransformer` in this demo and here we specify the cache folder. If you already downloaded the models in a local file system, set this folder here, otherwise the library tries to download the models in this folder if not available locally.

In particular, these models will be downloaded if not present in the cache folder:

models/models--sentence-transformers--all-MiniLM-L6-v2


In [62]:
#setting the local downloaded sentence transformer models f
os.environ["TRANSFORMERS_CACHE"] = f"{parent_directory}/models"

In [63]:
from sentence_transformers import SentenceTransformer

# load model - may take a minute or two to download the first time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder=os.getenv("TRANSFORMERS_CACHE", f"{parent_directory}/models"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [64]:
%%time

# create embeddings
chunk_embeddings = model.encode([chunk.page_content for chunk in chunks])
len(chunk_embeddings) == len(chunks)

CPU times: user 586 ms, sys: 115 ms, total: 701 ms
Wall time: 1.59 s


True

In [65]:
print(f"embedding dim should be {len(chunk_embeddings[0])}")

embedding dim should be 384


### Set up some helper functions

Helper functions to encode the single query vector and display redis search results

In [66]:
import pandas as pd


def encode_one(input):
    return model.encode(input).astype(np.float32).tobytes()


def table_view(res):
    if res.total == 0:
        print("No documents found.")
        return None
    
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    return res_df


### Define a schema and create an index
Below we connect to Redis and create an index for vector search that contains a single text field and vector field.

In [67]:
from redis.commands.search.field import TagField, TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


index_name = "redispy"
key_prefix = f"doc:{index_name}"


def create_index(index_type: str = "FLAT"):       # Creates a FLAT index by default
    try:
        # check to see if index exists
        client.ft(index_name).info()
        print("Index already exists!")
    except:
        # define schema
        schema = (
            TagField("doc_id"),                    # Tag Field - synthetic ID
            TextField("content"),                  # Text Field
            VectorField("chunk_vector",            # Vector Field
                index_type, {                      # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",
                    "DIM": 384,                    # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.HASH)

        # create Index
        client.ft(index_name).create_index(fields=schema, definition=definition)

In [68]:
# Create the index
create_index()

In [69]:
# Check the info related to the newly created index
client.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'doc_id',
   b'attribute',
   b'doc_id',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b','],
  [b'identifier',
   b'content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   384,
   b'distance_metric',
   b'COSINE']],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.00818634033203125',
 'total_inverted_index_blocks': '0',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': 'nan',
 'bytes_per_record_avg':

### Process and load data using Redis
Below we use a Redis pipeline (not a transaction) to batch send writes to Redis. This method helps with throughput significantly. The batch_size param can be customized and benchmarked on your hardware and with your data. We typically recommend starting small (100-200) and increasing as needed.

In [70]:
# load expects an iterable of dictionaries

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        data = {
            'doc_id': f"{i}",
            'content': chunk.page_content,
            # For HASH -- must convert embeddings to bytes
            'chunk_vector': np.array(chunk_embeddings[i]).astype(np.float32).tobytes()
        }
        pipe.hset(f"{key_prefix}:{i}", mapping=data)
        # execute in "mini batches"
        if i % batch_size == 0:
            res = pipe.execute()

    # cleanup final batch execution
    res = pipe.execute()

In [71]:
# check the data size in Redis
len(chunks) == client.dbsize()

True

In [72]:
client.hgetall(f"{key_prefix}:0")

{b'chunk_vector': b'\x06t\xb3\xbc\x95\xe0\xf9\xbd\xdf@\xdb;K\x1b\x8c\xbb;7J<\xcb(\xb1=\xf8\xa6\xe3<3\xcfP\xbc\xf9\xf0J<`ys\xbd\xc2\xfd$=P\xad\xc8=TB\xad\xbd\x1d\xd9\xa3\xbd\x95\xb4\x9b<G\xccU=\xa6B\xee\xbcm~\x08\xbd\xd7\xd3i\xbd\x17\xf9X\xbc\x06\xb1C=m\x85\xa3\xbd\x1aC\x80<\n\xc5\x13=>x\x12\xbd@}\x80\xbc\x16\xe9\x81;D=\xb0=\x0b\x8c\xa7;,\t\xf5\xbcxo,\xbd\xcf\xcb\xaa\xbc\xc7\x17\xec=\xd8\xc3\xc0=t^\xc4=\x13,\xf29\x03\x96j=\x07\xd7\xd5\xbc\xfa\x04*\xbcz\x0f\x0e=\\\x1e\xe0<\x97N\xc3\xbd\xbcD\xb0\xbd5;>=\xad\xb3\xd8\xbc\xcauH=\xe3\x1e\x08\xbdB`\x03=B],=\xb7v\xb7=S\xdbb\xbc\xd0$\xd3\xbc86\x85=\xf1\xe5\x1b\xbd\xce\xdez\xbc\x87\x11\xe3;a\xd9$\xbb\xf1\xaa\xa3=Xs\xa9<\x06j\x8e<\xc6\x82\xe9=\xbb\x1e\x91\xbd^\xc5)\xbd\xe3\x0b\xca<\xac\xb7\x8c;\x9f\xe4k<6=Y\xbcE-A\xbcW\xd4\xfb<\x0c9\xe8\xbd\xaa\xbf-=[\xcc8\xbc\xf9\xba9\xbdK\xca\xb9=\xc3\xees\xbd\xc4\xff\xd1<7\x87\xbb\xbcT\x9c\xaa=,\xed}<\x96I\xbd\xbcq3_\xbd\x9b\xd3\x84<9?>\xb9H\xc3/=hWe<\x9a\x89\x0e<\xbcx\xca\xbc\x12sH=\xc3\x91\x03=\x88\x9d\x05=\x

### Query the database
Now we can use the Redis search index to perform vector similarity search operations.

The code below takes a user input, converts to embeddings, and fetches the top 2 most semantically similar chunks from Redis.

In [73]:
# Grab user input
_input = "Nike profit margins and company performance"

query = (
    Query("*=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": np.array(model.encode(_input), dtype=np.float32).tobytes()
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,content
0,doc:redispy:85,0.321347296238,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."
1,doc:redispy:84,0.328033983707,As discussed in Note 15 — Operating Segments a...


In [74]:
# Example of sorting by a field other than vector_distance
query = (
    Query("*=>[KNN 4 @chunk_vector $vector as vector_distance]")
     .sort_by("doc_id")
     .return_fields("doc_id", "content", "vector_distance")
     .paging(0, 4)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,doc_id,content
0,doc:redispy:118,0.358749687672,118,"NIKE, INC. CONSOLIDATED STATEMENTS OF INCOME\n..."
1,doc:redispy:158,0.360825479031,158,Tax (expense) benefit Gain (loss) net of tax\n...
2,doc:redispy:84,0.328033983707,84,As discussed in Note 15 — Operating Segments a...
3,doc:redispy:85,0.321347296238,85,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."


### Range Queries
Range queries allow you to set a pre defined "threshold" for which we want to return documents

In [75]:
query = (
    Query("@chunk_vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

,id,vector_distance,content
0,doc:redispy:85,0.321347296238,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."
1,doc:redispy:84,0.328033983707,As discussed in Note 15 — Operating Segments a...
2,doc:redispy:118,0.358749687672,"NIKE, INC. CONSOLIDATED STATEMENTS OF INCOME\n..."
3,doc:redispy:158,0.360825479031,Tax (expense) benefit Gain (loss) net of tax\n...
4,doc:redispy:81,0.363789796829,"NIKE Brand revenues, which represented over 90..."
5,doc:redispy:82,0.370122373104,"Lower margin in our NIKE Direct business, driv..."
6,doc:redispy:80,0.386211693287,"4,780 (508)\n\n7 % -80 %\n\nTOTAL NIKE BRAND W..."
7,doc:redispy:86,0.393618762493,Apparel revenues increased 9% on a currency-ne...
8,doc:redispy:89,0.39476788044,"3 % -4 %\n\n13 % 4 %\n\n1,494 190\n\n8 % 23 %\..."
9,doc:redispy:159,0.395310223103,ASIA PACIFIC & LATIN AMERICA\n\n(1)\n\nGLOBAL ...


### Add filter statements
Redis queries can contain both vector search and traditional filters (numeric, tags, text, geo) in one single command.

In [76]:
# filter for docs that contain "profit" in the content field and do KNN vector search
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

,id,vector_distance,content
0,doc:redispy:81,0.363789796829,"NIKE Brand revenues, which represented over 90..."
1,doc:redispy:79,0.616193056107,"(Dollars in millions, except per share data)\n..."


In [77]:
# lets clean up our index
client.ft(index_name).dropindex(True)

b'OK'

### What about JSON Support?

Redis also allows you to store data in JSON objects. The JSON fields can contain metadata and vectors. Below is a simple example of indexing JSON data.

In [78]:
# schema
schema = (
    TextField("$.content",                     # Text Field (JSON path)
        as_name="content"                      # Text Field Alias -- required for JSON
    ),
    VectorField("$.chunk_vector",              # Vector Field (JSON path)
        "FLAT", {                              # Vector Index Type: FLAT or HNSW
            "TYPE": "FLOAT32",
            "DIM": 384,                        # Number of Vector Dimensions
            "DISTANCE_METRIC": "COSINE",       # Vector Search Distance Metric
        },
        as_name="chunk_vector"                 # Vector Field Alias -- required for JSON
    ),
)

# index Definition
definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.JSON) # select JSON here

# create Index
client.ft(index_name).create_index(fields=schema, definition=definition)

b'OK'

In [79]:
client.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'JSON',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'$.content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'$.chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   384,
   b'distance_metric',
   b'COSINE']],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.00818634033203125',
 'total_inverted_index_blocks': '0',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': 'nan',
 'bytes_per_record_avg': 'nan',
 'offsets_per_term_avg': 'nan',
 'offset_bits_per_record_avg': 'nan',
 'hash_indexing_failures': '0',

In [80]:
# Write JSON data to the index

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        redis_key = f"{key_prefix}:{i}"
        data = {
            'content': chunk.page_content,
            'chunk_vector': chunk_embeddings[i].tolist() # notice that we don't need to convert JSON embeddings to bytes
        }
        #print(data)
        pipe.json().set(redis_key, "$", data)
        # mini batch
        if i % batch_size == 0:
            res = pipe.execute()

    res = pipe.execute() # make sure to use mini batches if working with larger datasets

In [81]:
# Fetch the JSON doc
client.json().get(f"{key_prefix}:0", "$")

[{'content': "NIKE, Inc.(Exact name of Registrant as specified in its charter)Oregon93-0584541(State or other jurisdiction of incorporation)(IRS Employer Identification No.)One Bowerman Drive, Beaverton, Oregon 97005-6453(Address of principal executive offices and zip code)(503) 671-6453(Registrant's telephone number, including area code)SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:Class B Common StockNKENew York Stock Exchange(Title of each class)(Trading symbol)(Name of each exchange on which registered)SECURITIES REGISTERED PURSUANT TO SECTION 12(G) OF THE ACT:NONE",
  'chunk_vector': [-0.021905910223722454,
   -0.12201038748025894,
   0.00669108284637332,
   -0.004275714512914419,
   0.012342269532382488,
   0.08650358766317368,
   0.027789577841758728,
   -0.012744712643325329,
   0.012386553920805454,
   -0.05944192409515381,
   0.04028106480836868,
   0.09798681735992432,
   -0.08459916710853577,
   -0.08000395447015762,
   0.0190070066601038,
   0.0521967671811580

In [82]:
# And now you can perform the same kinds of queries
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)

}
res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,content
0,doc:redispy:81,0.363789796829,"NIKE Brand revenues, which represented over 90..."
1,doc:redispy:79,0.616193056107,"(Dollars in millions, except per share data)\n..."


## Cleanup
Clean up the index and data.

In [83]:
client.ft(index_name).dropindex(True)

b'OK'